In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [40]:
Emp = pd.read_csv("employee_survey_data.csv")
Mgr = pd.read_csv("manager_survey_data.csv")
Gen = pd.read_csv("general_data.csv")
Tin = pd.read_csv("in_out_time/in_time.csv")
Tou = pd.read_csv("in_out_time/out_time.csv")

In [41]:
Emp = Emp.rename(columns={
    "EnvironmentSatisfaction": "Emp_EnvironmentSatisfaction",
    "JobSatisfaction": "Emp_JobSatisfaction",
    "WorkLifeBalance": "Emp_WorkLifeBalance"
})

Mgr["JobInvolvement"] = Mgr["JobInvolvement"].astype(float)
Mgr["PerformanceRating"] = Mgr["PerformanceRating"].astype(float)
Mgr = Mgr.rename(columns={
    "JobInvolvement": "Mgr_JobInvolvement",
    "PerformanceRating": "Mgr_PerformanceRating"
})

EmpMgr = Emp.merge(Mgr, on="EmployeeID", how="left")
EmpMgr[EmpMgr.columns[1:]].describe()


,Emp_EnvironmentSatisfaction,Emp_JobSatisfaction,Emp_WorkLifeBalance,Mgr_JobInvolvement,Mgr_PerformanceRating
count,4385.000000,4390.000000,4372.000000,4410.000000,4410.000000
mean,2.723603,2.728246,2.761436,2.729932,3.153741
std,1.092756,1.101253,0.706245,0.711400,0.360742
min,1.000000,1.000000,1.000000,1.000000,3.000000
25%,2.000000,2.000000,2.000000,2.000000,3.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,3.000000,3.000000,3.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000


In [42]:
GenEmpMgr = Gen.merge(EmpMgr, on="EmployeeID", how="left")
GenEmpMgr.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Emp_EnvironmentSatisfaction,Emp_JobSatisfaction,Emp_WorkLifeBalance,Mgr_JobInvolvement,Mgr_PerformanceRating
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,6,1,0,0,3.0,4.0,2.0,3.0,3.0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,6.0,3,5,1,4,3.0,2.0,4.0,2.0,4.0
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,5.0,2,5,0,3,2.0,2.0,1.0,3.0,3.0
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,13.0,5,8,7,5,4.0,4.0,3.0,2.0,3.0
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,9.0,2,6,0,4,4.0,1.0,3.0,3.0,3.0


In [43]:
Tou = Tou.rename(columns={"Unnamed: 0": "EmployeeID"})
Tin = Tin.rename(columns={"Unnamed: 0": "EmployeeID"})

Tin_long = Tin.melt(
    id_vars="EmployeeID",
    var_name="Date",
    value_name="InTime"
)

Tou_long = Tou.melt(
    id_vars="EmployeeID",
    var_name="Date",
    value_name="OutTime"
)

time_long = Tin_long.merge(
    Tou_long,
    on=["EmployeeID", "Date"],
    how="outer"  
)

time_long["Date"] = pd.to_datetime(time_long["Date"], errors="coerce")
time_long["InTime"] = pd.to_datetime(time_long["InTime"], errors="coerce")
time_long["OutTime"] = pd.to_datetime(time_long["OutTime"], errors="coerce")

time_long["HoursWorked"] = (
    time_long["OutTime"] - time_long["InTime"]
) / pd.Timedelta(hours=1)

time_long = time_long.reset_index(drop=True)

# Make sure Date is datetime
time_long["Date"] = pd.to_datetime(time_long["Date"], errors="coerce")

# Add week and month
time_long["Week"] = time_long["Date"].dt.to_period("W").astype(str)
time_long["Month"] = time_long["Date"].dt.to_period("M").astype(str)

# (optional) reset index to make it pretty
time_long = time_long.reset_index(drop=True)

time_long.head(20)


,EmployeeID,Date,InTime,OutTime,HoursWorked,Week,Month
0,1,2015-01-01,NaT,NaT,NaN,2014-12-29/2015-01-04,2015-01
1,1,2015-01-02,2015-01-02 09:43:45,2015-01-02 16:56:15,7.208333,2014-12-29/2015-01-04,2015-01
2,1,2015-01-05,2015-01-05 10:08:48,2015-01-05 17:20:11,7.189722,2015-01-05/2015-01-11,2015-01
3,1,2015-01-06,2015-01-06 09:54:26,2015-01-06 17:19:05,7.410833,2015-01-05/2015-01-11,2015-01
4,1,2015-01-07,2015-01-07 09:34:31,2015-01-07 16:34:55,7.006667,2015-01-05/2015-01-11,2015-01
5,1,2015-01-08,2015-01-08 09:51:09,2015-01-08 17:08:32,7.289722,2015-01-05/2015-01-11,2015-01
6,1,2015-01-09,2015-01-09 10:09:25,2015-01-09 17:38:29,7.484444,2015-01-05/2015-01-11,2015-01
7,1,2015-01-12,2015-01-12 09:42:53,2015-01-12 16:58:39,7.262778,2015-01-12/2015-01-18,2015-01
8,1,2015-01-13,2015-01-13 10:13:06,2015-01-13 18:02:58,7.831111,2015-01-12/2015-01-18,2015-01
9,1,2015-01-14,NaT,NaT,NaN,2015-01-12/2015-01-18,2015-01


In [44]:
cols_to_drop = ["Over18", "EmployeeCount", "StandardHours"]

GenEmpMgr = GenEmpMgr.drop(
    columns=[c for c in cols_to_drop if c in GenEmpMgr.columns]
)

GenEmpMgr.to_csv("Employees.csv", index=False)
time_long.to_csv("Worktime.csv", index=False)


In [45]:
# --- Load ---
Worktime = pd.read_csv("Worktime.csv")

# --- Parse types ---
Worktime["Date"] = pd.to_datetime(Worktime["Date"], errors="coerce")
Worktime["InTime"] = pd.to_datetime(Worktime["InTime"], errors="coerce")
Worktime["OutTime"] = pd.to_datetime(Worktime["OutTime"], errors="coerce")

# --- HoursWorked (recompute or coerce) ---
if "HoursWorked" not in Worktime.columns:
    Worktime["HoursWorked"] = (Worktime["OutTime"] - Worktime["InTime"]) / pd.Timedelta(hours=1)
else:
    Worktime["HoursWorked"] = pd.to_numeric(Worktime["HoursWorked"], errors="coerce")

# --- Basic cleanup ---
Worktime.loc[(Worktime["HoursWorked"] < 0) | (Worktime["HoursWorked"] > 18), "HoursWorked"] = np.nan

# --- Time helpers ---
Worktime["WeekStart"] = Worktime["Date"].dt.to_period("W").dt.start_time  # Monday 00:00
Worktime["StartMin"] = (
    Worktime["InTime"].dt.hour * 60
    + Worktime["InTime"].dt.minute
    + Worktime["InTime"].dt.second / 60
)

# --- Flags ---
Worktime["IsMissingDay"] = Worktime["HoursWorked"].isna()
Worktime["IsLateArrival"] = Worktime["StartMin"].gt(10 * 60)  # after 10:00

# =========================================================
# 1) Daily -> Weekly aggregation (THIS creates `weekly`)
# =========================================================
weekly = (
    Worktime.groupby(["EmployeeID", "WeekStart"], as_index=False)
    .agg(
        week_hours=("HoursWorked", "sum"),
        days_logged=("Date", "nunique"),
        late_days=("IsLateArrival", "sum"),
        start_time_std=("StartMin", "std"),
        valid_hw=("HoursWorked", lambda s: s.notna().sum()),
    )
)

# Weeks with no valid hours should be NaN, not 0
weekly.loc[weekly["valid_hw"].eq(0), "week_hours"] = np.nan
weekly = weekly.drop(columns=["valid_hw"])

# =========================================================
# 2) Trend slope helper
# =========================================================
def slope_over_time(x: pd.Series) -> float:
    y = x.to_numpy(dtype=float)
    mask = np.isfinite(y)
    if mask.sum() < 2:
        return np.nan
    y = y[mask]
    t = np.arange(len(x), dtype=float)[mask]
    t = t - t.mean()
    y = y - y.mean()
    denom = np.dot(t, t)
    return float(np.dot(t, y) / denom) if denom != 0 else np.nan

# =========================================================
# 3) Employee-level features
# =========================================================
weekly = weekly.sort_values(["EmployeeID", "WeekStart"])

def build_features(dfw: pd.DataFrame) -> pd.Series:
    wh = dfw["week_hours"]

    last4 = dfw.tail(4)
    prev4 = dfw.iloc[max(len(dfw)-8, 0):max(len(dfw)-4, 0)]

    mean_last4 = last4["week_hours"].mean()
    mean_prev4 = prev4["week_hours"].mean()
    delta_recent = mean_last4 - mean_prev4

    start_std_mean = dfw["start_time_std"].mean()

    wh_mean = wh.mean()
    wh_std  = wh.std()
    wh_cv   = (wh_std / wh_mean) if (pd.notna(wh_mean) and wh_mean != 0) else np.nan

    return pd.Series({
        "weekly_hours_mean": wh_mean,
        "weekly_hours_max": wh.max(),
        "weekly_hours_std": wh_std,
        "weekly_hours_cv": wh_cv,
        "hours_trend_slope": slope_over_time(wh),
        "mean_hours_last_4w": mean_last4,
        "delta_recent_hours": delta_recent,
        "late_arrival_rate": (
            dfw["late_days"].sum() / dfw["days_logged"].sum()
            if dfw["days_logged"].sum() > 0 else np.nan
        ),
        "start_time_variability": start_std_mean,
    })

features = weekly.groupby("EmployeeID").apply(build_features).reset_index()
features.head()


C:\Users\pierr\AppData\Local\Temp\ipykernel_17984\2713818491.py:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  features = weekly.groupby("EmployeeID").apply(build_features).reset_index()


,EmployeeID,weekly_hours_mean,weekly_hours_max,weekly_hours_std,weekly_hours_cv,hours_trend_slope,mean_hours_last_4w,delta_recent_hours,late_arrival_rate,start_time_variability
0,1,32.277112,37.852500,6.542255,0.202690,0.010243,27.875625,-3.402014,0.432950,13.988352
1,2,34.371258,39.911944,6.553189,0.190659,0.042818,33.062014,1.623403,0.425287,16.906842
2,3,32.022720,36.631389,5.385144,0.168166,0.020674,30.027778,2.002361,0.501916,15.064614
3,4,31.896499,36.950278,6.335861,0.198638,0.032187,32.633056,2.114167,0.429119,15.547363
4,5,37.009675,41.326111,6.137144,0.165825,0.051585,36.201667,2.179722,0.432950,15.194054


In [46]:
# Load Employees
Employees = pd.read_csv("Employees.csv")

# Safety: ensure key exists
assert "EmployeeID" in features.columns
assert "EmployeeID" in Employees.columns

# Optional: avoid column name collisions (recommended)
Employees = Employees.add_prefix("emp_")
Employees = Employees.rename(columns={"emp_EmployeeID": "EmployeeID"})

# Merge (features is the reference table)
features_full = features.merge(
    Employees,
    on="EmployeeID",
    how="left"
)

features_full.head()


,EmployeeID,weekly_hours_mean,weekly_hours_max,weekly_hours_std,weekly_hours_cv,hours_trend_slope,mean_hours_last_4w,delta_recent_hours,late_arrival_rate,start_time_variability,...,emp_TotalWorkingYears,emp_TrainingTimesLastYear,emp_YearsAtCompany,emp_YearsSinceLastPromotion,emp_YearsWithCurrManager,emp_Emp_EnvironmentSatisfaction,emp_Emp_JobSatisfaction,emp_Emp_WorkLifeBalance,emp_Mgr_JobInvolvement,emp_Mgr_PerformanceRating
0,1,32.277112,37.852500,6.542255,0.202690,0.010243,27.875625,-3.402014,0.432950,13.988352,...,1.0,6,1,0,0,3.0,4.0,2.0,3.0,3.0
1,2,34.371258,39.911944,6.553189,0.190659,0.042818,33.062014,1.623403,0.425287,16.906842,...,6.0,3,5,1,4,3.0,2.0,4.0,2.0,4.0
2,3,32.022720,36.631389,5.385144,0.168166,0.020674,30.027778,2.002361,0.501916,15.064614,...,5.0,2,5,0,3,2.0,2.0,1.0,3.0,3.0
3,4,31.896499,36.950278,6.335861,0.198638,0.032187,32.633056,2.114167,0.429119,15.547363,...,13.0,5,8,7,5,4.0,4.0,3.0,2.0,3.0
4,5,37.009675,41.326111,6.137144,0.165825,0.051585,36.201667,2.179722,0.432950,15.194054,...,9.0,2,6,0,4,4.0,1.0,3.0,3.0,3.0


In [47]:
features_full = features_full.rename(columns={
    "EmployeeID": "employee_id",

    "weekly_hours_mean": "avg_weekly_hours",
    "weekly_hours_max": "max_weekly_hours",
    "weekly_hours_std": "weekly_hours_std",
    "weekly_hours_cv": "weekly_hours_cv",
    "hours_trend_slope": "weekly_hours_trend",
    "mean_hours_last_4w": "avg_weekly_hours_last_4w",
    "delta_recent_hours": "delta_weekly_hours_4w",
    "late_arrival_rate": "late_arrival_ratio",
    "start_time_std": "start_time_variability",

    "emp_Age": "age",
    "emp_Attrition": "attrition",
    "emp_BusinessTravel": "business_travel",
    "emp_Department": "department",
    "emp_DistanceFromHome": "distance_from_home",
    "emp_Education": "education_level",
    "emp_EducationField": "education_field",
    "emp_Gender": "gender",
    "emp_JobLevel": "job_level",
    "emp_JobRole": "job_role",
    "emp_MaritalStatus": "marital_status",
    "emp_MonthlyIncome": "monthly_income",
    "emp_NumCompaniesWorked": "num_companies_worked",
    "emp_PercentSalaryHike": "percent_salary_hike",
    "emp_StockOptionLevel": "stock_option_level",
    "emp_TotalWorkingYears": "total_working_years",
    "emp_TrainingTimesLastYear": "training_times_last_year",
    "emp_YearsAtCompany": "years_at_company",
    "emp_YearsSinceLastPromotion": "years_since_promotion",
    "emp_YearsWithCurrManager": "years_with_manager",

    "emp_Emp_EnvironmentSatisfaction": "environment_satisfaction",
    "emp_Emp_JobSatisfaction": "job_satisfaction",
    "emp_Emp_WorkLifeBalance": "work_life_balance",
    "emp_Mgr_JobInvolvement": "job_involvement",
    "emp_Mgr_PerformanceRating": "performance_rating",
})


In [48]:
features_full.to_csv("Features.csv", index=False)

In [49]:
features_full.dtypes


employee_id                   int64
avg_weekly_hours            float64
max_weekly_hours            float64
weekly_hours_std            float64
weekly_hours_cv             float64
weekly_hours_trend          float64
avg_weekly_hours_last_4w    float64
delta_weekly_hours_4w       float64
late_arrival_ratio          float64
start_time_variability      float64
age                           int64
attrition                    object
business_travel              object
department                   object
distance_from_home            int64
education_level               int64
education_field              object
gender                       object
job_level                     int64
job_role                     object
marital_status               object
monthly_income                int64
num_companies_worked        float64
percent_salary_hike           int64
stock_option_level            int64
total_working_years         float64
training_times_last_year      int64
years_at_company            

In [50]:
features_full["attrition"] = features_full["attrition"].map({"Yes": 1, "No": 0}).astype("Int64")

In [51]:
y = features_full["attrition"].astype(int)
X = features_full.drop(columns=["employee_id", "attrition"])

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

In [53]:
X_train.isna().mean().sort_values(ascending=False).head(20)

work_life_balance           0.007560
environment_satisfaction    0.005745
job_satisfaction            0.004838
num_companies_worked        0.004838
total_working_years         0.002117
avg_weekly_hours            0.000000
job_role                    0.000000
job_involvement             0.000000
years_with_manager          0.000000
years_since_promotion       0.000000
years_at_company            0.000000
training_times_last_year    0.000000
stock_option_level          0.000000
percent_salary_hike         0.000000
monthly_income              0.000000
marital_status              0.000000
job_level                   0.000000
max_weekly_hours            0.000000
gender                      0.000000
education_field             0.000000
dtype: float64

In [ ]:
num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ]
)

log_reg = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=2000))
])


In [ ]:
log_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['avg_weekly_hours', 'max_weekly_hours', 'weekly_hours_std',
       'weekly_hours_cv', 'weekly_hours_trend', 'avg_weekly_hours_last_4w',
       'delta_weekly_hours_4w', 'late_arrival_ratio', 'start_time_var...
       'environment_satisfaction', 'job_satisfaction', 'work_life_balance',
       'job_involvement', 'performance_rating'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['business_travel', 'department', 'education_field', 'gender',
       'job_role', 'marital_status'],
      dtype='object'))])),
                ('model', LogisticRegression(max_iter=2000))])

In [57]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


              precision    recall  f1-score   support

           0       0.87      0.97      0.92       925
           1       0.63      0.25      0.35       178

    accuracy                           0.85      1103
   macro avg       0.75      0.61      0.64      1103
weighted avg       0.83      0.85      0.83      1103

ROC AUC: 0.8159246887336775
